In [1]:
import numpy as np
from gurobipy import *
import matplotlib.pyplot as plt
import pandas as pd
from itertools import combinations
import matplotlib.lines as lines

In [2]:
def Generate_Instance(num_cities):
    city_list = [np.random.uniform(0,10,2) for city in range(num_cities)]
    student_list = [0] + [np.random.randint(10,50) for city in range(1,num_cities)]

    return city_list, student_list

def Get_Distance_Matrix(city_list):
    
    num_cities = len(city_list)
    distance_matrix = [[0 for j in range(num_cities)] for i in range(num_cities)]
    for i in range(num_cities):
        for j in range(num_cities):
            distance  = ((city_list[i][0] - city_list[j][0])**2 + (city_list[i][1] - city_list[j][1])**2)**0.5
            distance_matrix[i][j] = distance
    
    
    return distance_matrix

Solve the Bus Routing problem from Lecture 12.  Return the optimal objective value!

In [3]:


def Bus_Routing(num_cities, city_list, student_list, distance_matrix, K_1, C_1, K_2, C_2, Q):
    
    m = Model()
    x, y, z, q, u, ts, tsk = {}, {}, {}, {}, {}, {}, {}
    K_total = K_1 + K_2
    M = 10000

    for k in range(K_total):
        for i in range(1, num_cities):
            ts[i] = m.addVar(0,GRB.INFINITY,0,GRB.CONTINUOUS)
            tsk[(i, k)] = m.addVar(0,GRB.INFINITY,0,GRB.CONTINUOUS)
            y[(i, k)] = m.addVar(0,1,0,GRB.BINARY)
            z[i] = m.addVar(0, 1, 0, GRB.BINARY)
            for j in range(1, num_cities):
                q[(i, j)] = m.addVar(0, 1 if distance_matrix[i][j] <= 2 else 0, 0, GRB.BINARY)
        for i in range(num_cities):
            u[(i, k)] = m.addVar(0,GRB.INFINITY,0,GRB.CONTINUOUS)
            for j in range(num_cities):
                if i != j:
                    x[(i, j, k)] = m.addVar(0,1,distance_matrix[i][j],GRB.BINARY)

    m.addConstr(quicksum(z[i] for i in range(1, num_cities)) == Q)

    for i in range(1, num_cities):
        m.addConstr(quicksum(q[(i, j)] for j in range(1, num_cities)) == 1)
        m.addConstr(quicksum(y[(i, k)] for k in range(K_total)) == z[i])
        for j in range(1, num_cities):
            m.addConstr(q[(i, j)] <= z[j])

    for k in range(K_total):
        for i in range(num_cities):
            if i == 0:
                m.addConstr(quicksum(x[(i, j, k)] for j in range(num_cities) if i != j) == 1)
                m.addConstr(quicksum(x[(j, i, k)] for j in range(num_cities) if i != j) == 1)
            else:
                m.addConstr(quicksum(x[(i, j, k)] for j in range(num_cities) if i != j) == y[(i, k)])
                m.addConstr(quicksum(x[(j, i, k)] for j in range(num_cities) if i != j) == y[(i, k)])
        for i in range(1, num_cities):
            m.addConstr(quicksum(q[(j, i)] * student_list[j] for j in range(1, num_cities)) == ts[i])
            m.addConstr(tsk[(i, k)] >= ts[i] - M * (1 - y[(i, k)]))
            for j in range(1, num_cities):
                if i != j:
                    m.addConstr(u[(i, k)] + distance_matrix[i][j] <= u[(j, k)] + M * (1 - x[(i, j, k)]))

    for k in range(K_total):
        if k < K_1:
            m.addConstr(quicksum(tsk[(i, k)] for i in range(1, num_cities)) <= C_1)
        else:
            m.addConstr(quicksum(tsk[(i, k)] for i in range(1, num_cities)) <= C_2)

    m.setParam("OutputFlag", False)
    m.optimize()
    return m.objVal




In [4]:
np.random.seed(100)
num_cities = 11
K_1=2
C_1=175
K_2=2
C_2=125
Q=8
city_list, student_list = Generate_Instance(num_cities)
distance_matrix = Get_Distance_Matrix(city_list)
obj = Bus_Routing(num_cities, city_list, student_list, distance_matrix, K_1,C_1, K_2,C_2,Q)

assert np.isclose(obj,39.5118)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-12


In [5]:
np.random.seed(100)
num_cities =21
K_1=2
C_1=200
K_2=2
C_2=150
Q=10
city_list, student_list = Generate_Instance(num_cities)
distance_matrix = Get_Distance_Matrix(city_list)
obj = Bus_Routing(num_cities, city_list, student_list, distance_matrix, K_1,C_1, K_2,C_2,Q)

assert np.isclose(obj,46.1440)

In [6]:
#this one may take a minute or two to run
np.random.seed(100)
num_cities =31
K_1=1
C_1=200
K_2=3
C_2=500
Q=13
city_list, student_list = Generate_Instance(num_cities)
distance_matrix = Get_Distance_Matrix(city_list)
obj = Bus_Routing(num_cities, city_list, student_list, distance_matrix, K_1,C_1, K_2,C_2,Q)

assert np.isclose(obj,48.04449)